In [1]:
pwd

'/Users/moustaphakebe1998gmail.com/Documents/TestDeep/ChatBot-deeplearning'

In [2]:
ls

chat_model.keras      label_encoder.pickle  tokenizer.pickle
intents.json          requirements.txt      train.ipynb


In [3]:
! pip install -r requirements.txt -q

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      Running from numpy source directory.
      <string>:461: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      Traceback (most recent call last):
        File "/Users/moustaphakebe1998gmail.com/Documents/TestDeep/env/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/Users/moustaphakebe1998gmail.com/Documents/TestDeep/env/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/moustaphakebe1998gmail.com/Documents/TestDeep/env/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_pr

In [4]:
!pip install pandas numpy -q

In [5]:
cat requirements.txt

tensorflow
nltk==3.5
colorama==0.4.3
numpy==1.18.5
scikit_learn==0.23.2
Flask==1.1.2

In [6]:
pip install tensorflow nltk colorama scikit_learn

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 2.0 MB/s eta 0:00:00a 0:00:01
Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl (23.1 MB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, MaxPooling1D, Conv1D, BatchNormalization
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [8]:
pwd

'/Users/moustaphakebe1998gmail.com/Documents/TestDeep/ChatBot-deeplearning'

In [9]:
ls

chat_model.keras      label_encoder.pickle  tokenizer.pickle
intents.json          requirements.txt      train.ipynb


In [10]:
with open("intents.json") as f:
    data= json.load(f)

In [11]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ['I.m Joana, your bot assistant',
    "I'm Joana, an Artificial Intelligent bot"]},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Joana.',
    "I'm Joana!",
    'Just call me as Joana']},
  {'tag': 'help',
   'patterns': ['Could you help me?',
    'give me a hand please',
    'Can you help

In [12]:
train_sentences=[]
train_label=[]
sentences=[]
label=[]
responses=[]
for intents in  data["intents"]:
    for pattern in intents["patterns"]:        
        train_sentences.append(pattern)
        train_label.append(intents["tag"])
    responses.append(intents["responses"])


    if intents["tag"] not in label:
        label.append(intents["tag"])
num_classes = len(label)  

In [13]:
num_classes

8

In [14]:
len(train_sentences)

33

In [15]:
import numpy as np
np.unique(train_label)

array(['about', 'complaint', 'createaccount', 'goodbye', 'greeting',
       'help', 'name', 'thanks'], dtype='<U13')

In [16]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(train_label)

LabelEncoder()

In [17]:
lbl_encoder.classes_

array(['about', 'complaint', 'createaccount', 'goodbye', 'greeting',
       'help', 'name', 'thanks'], dtype='<U13')

In [18]:
training_labels = lbl_encoder.transform(train_label)
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [19]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Global pooling
model.add(GlobalAveragePooling1D())

# Couches cachées avec Batch Normalization
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
#model.add(Dense(14, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dense(12, activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))

# Couche de sortie
model.add(Dense(num_classes, activation='softmax'))

# Compilation du modèle
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

/Users/moustaphakebe1998gmail.com/Documents/TestDeep/env/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
epochs = 70
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0919 - loss: 2.1792
Epoch 2/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3876 - loss: 1.8431  
Epoch 3/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4593 - loss: 1.7496  
Epoch 4/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5511 - loss: 1.6749  
Epoch 5/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5511 - loss: 1.6154  
Epoch 6/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6736 - loss: 1.5560
Epoch 7/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7042 - loss: 1.4820  
Epoch 8/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7042 - loss: 1.4341  
Epoch 9/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7244 - loss: 1.3989 
Epoch 10/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7348 - loss: 1.3371 
Epoch 11/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7961 - loss: 1.3196  
Epoch 12/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7655 - loss: 1.28

In [22]:
# Étape 1 : Prétraitement
text = "help me"
# Exemple de transformation : convertis en minuscules
text = text.lower()

# Étape 2 : Tokenisation
# Assumons que `tokenizer` est le tokenizer utilisé lors de l'entraînement
sequence = tokenizer.texts_to_sequences([text])

# Étape 3 : Paddage
# Assumons que `max_length` est la longueur utilisée lors de l'entraînement
padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')

# Étape 4 : Prédiction
prediction = model.predict(padded_sequence)

# Optionnel : Affiche la prédiction
print("Prediction:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Prediction: [[0.11265241 0.12379357 0.13620123 0.13648829 0.1077271  0.13652338
  0.10362844 0.14298555]]


In [23]:
predicted_index = np.argmax(prediction)

# Convertir l'indice en étiquette lisible
tag = lbl_encoder.inverse_transform([predicted_index])

In [24]:
print(tag[0])

thanks


In [25]:
with open("intents.json") as file:
    data = json.load(file)

In [26]:
for i in data['intents']:
    if i['tag'] == tag:
        print(np.random.choice(i['responses']))

Any time!


In [27]:
# to save the trained model
model.save("chat_model.keras")

In [28]:

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model.keras')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()